#### 0. saspy

In [25]:
import saspy
import numpy as np
import pandas as pd
from IPython.display import HTML

In [60]:
sascfg="c:/code/pub/ct/sascfg.py"
sas=saspy.SASsession(cfgfile=sascfg)
sas.HTML_Style="PowerPointDark"
resp=sas.submit("""%let ROOT=/home/u63544628;""")

Using SAS Config named: oda
SAS Connection established. Subprocess id is 11868



In [84]:
resp=sas.submit("""
proc import datafile="&ROOT./bl.csv" out=bl dbms=csv replace;
    getnames=yes;
run;

data bl_; set bl;
    format TYPE $3. RESULT 1. TYPEX $10. DATES yymmn6.;
    DATEX=put(datepart(DATE), date9.);
    RESULT=put(substr(RESULT,1,1),1.);

    if BATCH="MAS" then TYPE="MAC";
    else if BATCH="AAS" then TYPE=.;
    else TYPE="#NA";

    array col TYPE;
    do over col; if col=. then col="#NA";
    end;

    TYPEX=put(cats(BATCH,"-",LOCALE,"-",TYPE),$10.);
    YEAR=year(DATE);
    MONTH=month(DATE);
    DAY=day(DATE);
    DATES=DATE;
    DAYSSINCE=intck("day",DATE,today());

    if cmiss(of _all_) then delete;
run;

proc sort data=bl_ out=bl_ nodupkey;
    by BATCH LOCALE;
run;

proc print data=bl_(obs=10) noobs; run;
""")
HTML(resp["LST"])

BATCH,LOCALE,INPUT,OUTPUT,DATE,RESULT,TYPE,TYPEX,DATES,DATEX,YEAR,MONTH,DAY,DAYSSINCE
AAS,KR,마이힐링타운,룰더스카이 (Rule the Sky),2023-07-02,2,#NA,AAS-KR-#NA,202307,01JAN1960,2023,7,2,62
MAS,KR,page2,Toolbox for Pages - 전문 디자인 템플릿,2023-06-30,5,#NA,MAS-KR-#NA,202306,01JAN1960,2023,6,30,64
